# My hangman file

In [2]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
import pandas as pd
import numpy as np
import torch

In [1]:
import torch
from torch import nn

import numpy as np

In [3]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


Test-train split on data

In [4]:
from sklearn.model_selection import train_test_split
dict_train, dict_test = train_test_split(full_dict)

In [5]:
n_epochs = 100
max_input_length = max([len(word) for word in dict_train])
batch_size = np.array([len(word) for word in dict_train]).mean()
print("Training set size: %i" % len(dict_train))
print("Max input size: %i" % max_input_length)
print("Average word length: %f" % batch_size)


Training set size: 170475
Max input size: 29
Average word length: 9.357941


Ok, so now what, we have input:

blanked word -> encode it
guess list -> encode it
remaining correct guesses *this is the target -> encode it?

how do I encode the blanked word though? the strings also have variable length...

Ok, convert word into numbers

In [9]:
longest_word_length = 45

In [10]:
def word_to_tensor(current_word):
    longest_word_length = 45 #this is the longest word in the english language
    #first, encode word: ex "apple"
    #assume word is something like "_pp__"
    alphabet = list(string.ascii_lowercase) 
    alphabet.append("_")#this is for blanks
    #alphabet.append("=") # using this for generic padding
    #input_length = longest_word_length + n_letters+1
    num_letters = len(alphabet)

    # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
    # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
    current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
    for widx, letter in enumerate(current_word):
        #print(widx)
        #print(letter)
        current_word_tensor[0][widx][alphabet.index(letter)] = 1
    #print(current_word_tensor)
    return current_word_tensor

def guess_list_to_tensor(guess_list):
    #need to encode the guess list as a 1 x 26 tensor
    guess_tensor = torch.zeros(1, 26)
    for letter in guess_list:
        guess_tensor[0][alphabet.index(letter)] = 1
    
    return guess_tensor

    

In [11]:
print(word_to_tensor("_pp__"))
print(guess_list_to_tensor(["p"]))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])


In [12]:
print(word_to_tensor("_pp__").shape)
print(guess_list_to_tensor(["p"]).shape)

torch.Size([1, 27, 45])
torch.Size([1, 26])


Ok, so the current word is converted into a tensor and then fed into a LSTM, the guess list is converted to a tensor and then COMBINED with the output from the LSTM and then that combo is fed into a dense layer.

In [13]:
import torch.nn as nn
import torch.optim as optim

class Player(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Player, self).__init__()
        self.num_classes = num_classes #number of outputs
        self.num_layers = num_layers #I think this is arbitrary
        self.input_size =  input_size #input features
        self.hidden_size = hidden_size #to match number of letters in encoded guess
        self.seq_length = seq_length #to match number of letters in input
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size,
                          num_layers=self.num_layers, dropout = True, bidirectional = True,batch_first=True) #lstm
        self.relu = nn.ReLU()
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, 26) #fully connected last layer
        self.learning_rate = 0.005
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)


    
    def forward(self, encoded_word, encoded_guess):
        h_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #hidden state
        c_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        #self.lstm.flatten_parameters()
        out, (hn, cn) = self.lstm(encoded_word, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = out[:,-1]
        #print(out.shape)
        #print('The shape of lstm_out is:', out.shape)
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        #print(hn.shape)
        #print(encoded_guess.shape)
        hn = torch.cat((hn,encoded_guess)) #combine data with encoded guess data before passing to dense layer



        out = self.fc_1(out) #first Dense
        #print(out.shape)

        out = self.fc(out) #Final Output
        #print(out.shape)
        out = self.softmax(out)
        return out
    
    def evaluate_loss(self, output, correct_responses, guess_list):




    

In [14]:
longest_word_length = 45
player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
#print(player1)
current_word = "_pp__"
guess_list = ["p"]
word_tensor = word_to_tensor(current_word)
guess_tensor = guess_list_to_tensor(guess_list)

player_output = player1(word_tensor, guess_tensor)

In [15]:
t1 = player_output.detach().numpy()[0]

In [16]:
t1

array([-3.2246401, -3.309238 , -3.232234 , -3.3280027, -3.3879778,
       -3.3917842, -3.2578187, -3.1014676, -3.2701752, -3.2820332,
       -3.1336472, -3.4049466, -3.3204627, -3.238895 , -3.166744 ,
       -3.2095344, -3.3126488, -3.0953586, -3.1537652, -3.259031 ,
       -3.318493 , -3.2638526, -3.191866 , -3.416239 , -3.2532258,
       -3.2842255], dtype=float32)

In [17]:
torch.argmax(player_output[0])

tensor(17)

In [18]:
def guess_from_output(output, guess_list):
    #output is the tensor I get from player
    output = output.detach().numpy()[0] #this is just easier to deal with

    alphabet = list(string.ascii_lowercase)
    idx = np.argmax(output)
    print(idx)
    guess = alphabet[idx]
    #I will here force the guess to be a new one...
    while guess in guess_list:
        output[idx] -= 1 
        idx = np.argmax(output)
        guess = alphabet[idx]


    return guess

In [19]:
def error_propagate(output, answer):
    #can use the guess_list_to_tensor to convert the answer word into a tensor
    answer_tensor = guess_list_to_tensor(list(answer))
    answer_tensor = answer_tensor.long()
    print(answer_tensor)
    loss = nn.NLLLoss()
    error = loss(torch.flatten(output), torch.flatten(answer_tensor))
    error.backward()    

In [20]:
error_propagate(player_output, "apple")

tensor([[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])


In [21]:
guess_list_to_tensor(list("apple"))

tensor([[1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]])

In [22]:
print(guess_from_output(player_output, guess_list))

17
r


Can almost start learning...


before start,
Initialize player object
Initialize hangman object

Each loop of training will:

generate correct word and set up game
zero out initial state of player



In [25]:
class MyHangman4(object):
    def __init__(self, player, dictionary):
        self.guessed_letters = []
        self.word = ""
        self.correct_word = ""
        self.full_dictionary = dictionary #going to use a subset of the training data obviously
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.has_won = False
        self.player = player
        self.learning_rate = 0.005 #should have this as an input?
        self.current_loss = 0
        
    #def generate_correct_word(self):
    #    self.correct_word = random.choice(self.full_dictionary)

    def get_winstate(self):
        return self.has_won
    
    #def add_player(self,player):
    #    self.player = player

    def word_to_tensor(current_word):
        longest_word_length = 45 #this is the longest word in the english language
        #first, encode word: ex "apple"
        #assume word is something like "_pp__"
        alphabet = list(string.ascii_lowercase) 
        alphabet.append("_")#this is for blanks
        #alphabet.append("=") # using this for generic padding
        #input_length = longest_word_length + n_letters+1
        num_letters = len(alphabet)

        # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
        # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
        current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
        for widx, letter in enumerate(current_word):
            #print(widx)
            #print(letter)
            current_word_tensor[0][widx][alphabet.index(letter)] = 1
        #print(current_word_tensor)
        return current_word_tensor

    def guess_list_to_tensor(self,guess_list):
        #need to encode the guess list as a 1 x 26 tensor
        guess_tensor = torch.zeros(1, 26)
        for letter in guess_list:
            guess_tensor[0][alphabet.index(letter)] = 1
        
        return guess_tensor


    def guess_from_output(self,output, guess_list):
        #output is the tensor I get from player
        output = output.detach().numpy()[0] #this is just easier to deal with

        alphabet = list(string.ascii_lowercase)
        idx = np.argmax(output)
        #print(idx)
        guess = alphabet[idx]
        #I will here force the guess to be a new one...
        while guess in guess_list:
            output[idx] -= 1 
            idx = np.argmax(output)
            guess = alphabet[idx]
        return guess
    
    def error_propagate(self, output, answer):
        #can use the guess_list_to_tensor to convert the answer word into a tensor
        answer_tensor = self.guess_list_to_tensor(list(answer))
        answer_tensor = answer_tensor.long()
        loss = nn.NLLLoss()
        #print(output.shape)
        #print(answer_tensor.shape)
        error = loss(torch.flatten(output), torch.flatten(answer_tensor))
        error.backward()    
        #record stuff so we can plot later
        for p in self.player.parameters():
            p.data.add(p.grad.data, alpha=-self.learning_rate)

        self.current_loss = error
        

    #everything gets put together in the guess function
    def guess(self, player, word, guess_list):
        word_tensor = word_to_tensor(word)
        guess_list_tensor = guess_list_to_tensor(guess_list)
        output = player(word_tensor, guess_list_tensor)
        self.error_propagate(output, self.correct_word)
        guess_letter = self.guess_from_output(output, guess_list)

        return guess_letter
    
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)

    def get_loss(self):
        return self.current_loss

    

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        self.has_won = False
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?

        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer
            self.player.zero_grad()

            #get guessed letter
            guess_letter = self.guess(self.player, word, self.guessed_letters)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                

In [26]:

learning_rate = 0.005
player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
game = MyHangman4(player1, dict_train)
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: __________.
Guessing letter: w
Incorrect guess w, # of tries remaining: 6. Word: __________.
Guessing letter: h
Incorrect guess h, # of tries remaining: 5. Word: __________.
Guessing letter: q
Incorrect guess q, # of tries remaining: 4. Word: __________.
Guessing letter: c
Got a Letter, c, # of tries remaining: 3. Word: c_________.
Guessing letter: s
Incorrect guess s, # of tries remaining: 3. Word: c_________.
Guessing letter: l
Got a Letter, l, # of tries remaining: 2. Word: c___l_____.
Guessing letter: t
Incorrect guess t, # of tries remaining: 2. Word: c___l_____.
Guessing letter: d
Incorrect guess d, # of tries remaining: 1. Word: c___l_____.
You Lose, the answer was: cupuliform


In [29]:
n_iters = 10
print_every = 1

player1 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
game = MyHangman4(player1, dict_train)

error_list = []
t = time.time()
elapsed = time.time() - t
wincount = 0

for i in range(1, n_iters+1):
    game.start_game()
    error_list.append(game.get_loss())
    wincount += game.get_winstate()

    #if(i % print_every == 0):
    #    elapsed = time.time() - t
    #    print("On Iteration %i of %i" % (i, n_iters))
    #    print("Current loss is: %f" % game.get_loss())
    #    print("Has been running for: %f seconds" % elapsed)
    

Successfully start a new game! # of tries remaining: 6. Word: _________.
Guessing letter: r
Got a Letter, r, # of tries remaining: 6. Word: r____r___.
Guessing letter: a
Incorrect guess a, # of tries remaining: 6. Word: r____r___.
Guessing letter: g
Incorrect guess g, # of tries remaining: 5. Word: r____r___.
Guessing letter: b
Incorrect guess b, # of tries remaining: 4. Word: r____r___.
Guessing letter: y
Incorrect guess y, # of tries remaining: 3. Word: r____r___.
Guessing letter: e
Got a Letter, e, # of tries remaining: 2. Word: re_e_r__e.
Guessing letter: z
Incorrect guess z, # of tries remaining: 2. Word: re_e_r__e.
Guessing letter: d
Got a Letter, d, # of tries remaining: 1. Word: rede_r__e.
Guessing letter: s
Incorrect guess s, # of tries remaining: 1. Word: rede_r__e.
You Lose, the answer was: redeprive
Successfully start a new game! # of tries remaining: 6. Word: ___________.
Guessing letter: r
Got a Letter, r, # of tries remaining: 6. Word: ___r_______.
Guessing letter: a
Got

# Why are the weights not changing?

Ok, I am going to completely take apart the wrapper for the game, and get rid of any references to accepting a Player object, just have it referenced as a thing that exists in global...

In [100]:
import torch.nn as nn
import torch.optim as optim

class Player(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(Player, self).__init__()
        self.num_classes = num_classes #number of outputs
        self.num_layers = num_layers #I think this is arbitrary
        self.input_size =  input_size #input features
        self.hidden_size = hidden_size #to match number of letters in encoded guess
        self.seq_length = seq_length #to match number of letters in input
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) #fully connected 1
        self.fc = nn.Linear(128, 26) #fully connected last layer
        self.softmax = nn.LogSoftmax(dim=1)


    
    def forward(self, encoded_word, encoded_guess):
        h_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #hidden state
        c_0 = torch.zeros(self.num_layers, encoded_word.size(0), self.hidden_size) #internal state
        # Propagate input through LSTM
        #self.lstm.flatten_parameters()
        out, (hn, cn) = self.lstm(encoded_word, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = out[:,-1]
        #print(out.shape)
        #print('The shape of lstm_out is:', out.shape)
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        #print(hn.shape)
        #print(encoded_guess.shape)
        hn = torch.cat((hn,encoded_guess)) #combine data with encoded guess data before passing to dense layer



        out = self.fc_1(out) #first Dense
        #print(out.shape)

        out = self.fc(out) #Final Output
        #print(out.shape)
        out = self.softmax(out)
        return out

In [126]:
player2 = Player(num_classes = len(alphabet), 
                 num_layers = 10,
                 input_size =  longest_word_length,
                 hidden_size = len(alphabet),
                 seq_length = 1)
player2.train()
optimizer = optim.Adam(player2.parameters(), lr = 0.05)

In [127]:


guessed_letters = []
#should include a self word and self correct word
word = ""
correct_word = ""

full_dictionary = dict_train #going to use a subset of the training data obviously       
current_dictionary = []
has_won = False
#learning_rate = 0.005 #should have this as an input?
learning_rate = 10
current_loss = 0

def word_to_tensor(current_word):
    longest_word_length = 45 #this is the longest word in the english language
    #first, encode word: ex "apple"
    #assume word is something like "_pp__"
    alphabet = list(string.ascii_lowercase) 
    alphabet.append("_")#this is for blanks
    #alphabet.append("=") # using this for generic padding
    #input_length = longest_word_length + n_letters+1
    num_letters = len(alphabet)

    # the current word tensor needs to be encoded as a 1 x 27 x N length tensor
    # 1 because we are using batches of 1, 27 because 26 letters + "_" and N is the max word length
    current_word_tensor = torch.zeros(1,num_letters,longest_word_length)
    for widx, letter in enumerate(current_word):
        #print(widx)
        #print(letter)
        current_word_tensor[0][widx][alphabet.index(letter)] = 1
    #print(current_word_tensor)
    return current_word_tensor

def guess_list_to_tensor(guess_list):
    #need to encode the guess list as a 1 x 26 tensor
    guess_tensor = torch.zeros(1, 26)
    for letter in guess_list:
        guess_tensor[0][alphabet.index(letter)] = 1
    
    return guess_tensor

def guess_from_output(output, guess_list):
    #output is the tensor I get from player
    output = output.clone().detach().numpy()[0] #this is just easier to deal with

    alphabet = list(string.ascii_lowercase)
    idx = np.argmax(output)
    #print(idx)
    guess = alphabet[idx]
    #I will here force the guess to be a new one...
    while guess in guess_list:
        output[idx] -= 1 
        idx = np.argmax(output)
        guess = alphabet[idx]
    return guess

def error_propagate(output, answer):
    #can use the guess_list_to_tensor to convert the answer word into a tensor
    answer_tensor = guess_list_to_tensor(list(answer))
    answer_tensor = answer_tensor.long()
    loss = nn.NLLLoss()
    #print(output.shape)
    #print(answer_tensor.shape)
    error = loss(torch.flatten(output), torch.flatten(answer_tensor))
    error.backward()    
    optimizer.step()
    optimizer.zero_grad()
    

#everything gets put together in the guess function
def guess(word, guess_list):
    word_tensor = word_to_tensor(word)
    guess_list_tensor = guess_list_to_tensor(guess_list)
    output = player2(word_tensor, guess_list_tensor)
    error_propagate(output, correct_word)
    guess_letter = guess_from_output(output, guess_list)

    return guess_letter

def generate_correct_word(self):
    self.correct_word = random.choice(self.full_dictionary)



def handle_guess(guessLetter, currentWord, ans):
    #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
    indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
    wordList = list(currentWord)
    for i in indices:
        wordList[i] = guessLetter #now I replace the occurrences of the letter
    word = "".join(wordList)
    return word
    
            
def play_game(verbose=True):
    has_won = False
    # reset guessed letters to empty set and current plausible dictionary to the full dictionary
    guessed_letters = []
    current_dictionary = full_dictionary

    #pick a word from the dictionary at random.
    correct_word = random.choice(full_dictionary)
    word = "_" * len(correct_word) #I should make this a member of self right?

    tries_remains = 6 #I think this needs to just be a magic number?

    if verbose:
        print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
    while tries_remains >0:
        #tries_remains -=1 tries remains decreases after WRONG answer

        #get guessed letter
        guess_letter = guess(word, guessed_letters)

        #append guessed letter to guessed letters field in hangman object
        guessed_letters.append(guess_letter)
        if verbose:
            print("Guessing letter: {0}".format(guess_letter))

        #apply guessed letter to the word
        new_word = handle_guess(guess_letter, word, correct_word) 

        #check if we have the word
        if new_word == correct_word:
            word = new_word
            if verbose:
                print("Success! the word was: %s" % word)
            has_won = True
            break
        
        if new_word == word:
            #this means that we did not get a correct guess
            #decrease number of tries
            if verbose:
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
            tries_remains -=1
        else:
            #We have a correct letter guessed, but not the complete word
            #don't decrement tries_remains
            word = new_word
            if verbose:
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
    if tries_remains == 0:
        if verbose:
            print("You Lose, the answer was: %s" % correct_word)
    
    return 
            

In [128]:
play_game()

Successfully start a new game! # of tries remaining: 6. Word: ___________.
Guessing letter: f
Incorrect guess f, # of tries remaining: 6. Word: ___________.
Guessing letter: a
Incorrect guess a, # of tries remaining: 5. Word: ___________.
Guessing letter: j
Incorrect guess j, # of tries remaining: 4. Word: ___________.
Guessing letter: m
Incorrect guess m, # of tries remaining: 3. Word: ___________.
Guessing letter: w
Incorrect guess w, # of tries remaining: 2. Word: ___________.
Guessing letter: h
Incorrect guess h, # of tries remaining: 1. Word: ___________.
You Lose, the answer was: squillgeing


In [67]:
optimizer.step()

In [68]:
player2.grad

AttributeError: 'Player' object has no attribute 'grad'

In [80]:
before_step = list(player2.parameters())[0].clone().detach().numpy()
optimizer.step()
after_step = list(player2.parameters())[0].clone().detach().numpy()
print(np.array_equal(before_step,after_step)) # Prints True

True


In [72]:
print(np.array_equal(before_step,after_step)) # Prints True

True


In [89]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.005
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [ ]:
class playerBasic():
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x


In [ ]:
class PlayerRNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(PlayerRNN, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden